In [ ]:
from utils.parse_openapi import parse_openapi_tools
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    openapi_json = json.load(f)

base_url = "http://8.218.206.87:8500"
tools = parse_openapi_tools(openapi_json, base_url)

# 查看工具信息
for name, info in tools.items():
    print(f"工具名: {name}")
    print(f"描述: {info['description']}")
    print(f"参数: {info['parameters']}")
    print("-" * 50)


In [ ]:
tool_name = "get_idea_by_need_get_idea_by_need_post"

if tool_name in tools:
    func = tools[tool_name]["func"]
    result = func(need="生成一个自动化脚本", limit=5)  # 参数按 JSON Schema 提供
    print("调用结果:", result)


In [ ]:
from utils.openapi_to_tools_json import openapi_to_mcp_tools
# 示例 openapi.json
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)

mcp_tools = openapi_to_mcp_tools(example_openapi)
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

In [9]:
# test_register_tools_request.py
import requests

# FastAPI 服务地址
BASE_URL = "http://127.0.0.1:8000"

# 构造最小 OpenAPI JSON 示例

with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)
payload = {
    "openapi_json": example_openapi,
    "base_url": "http://8.218.206.87:8500"
}

response = requests.post(f"{BASE_URL}/register_tools", json=payload)

print("状态码:", response.status_code)
print("返回内容:", response.json())


状态码: 200
返回内容: {'success': True, 'message': '成功注册 5 个工具到内存'}


In [16]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

payload = {
    "task": "你帮我测试一下获取某轮完整信息的工具可以用吗。你从第3轮开始查第一轮的数据，并且输出第一轮数据的完整"
}

response = requests.post(f"{BASE_URL}/run_agent", json=payload)

print("状态码:", response.status_code)
try:
    print("返回数据:", json.dumps(response.json(), ensure_ascii=False, indent=2))
except Exception:
    print("返回内容不是 JSON:", response.text)


状态码: 200
返回数据: {
  "result": "工具 `get_history_by_round` 可用。以下是第1轮的完整历史数据：\n\n```json\n{\n\"round\": 1,\n\"thought\": \"用户要求测试获取某轮完整信息的工具是否可用，具体是让我从第3轮开始查第一轮的数据，并输出第一轮数据的完整内容。我需要调用 `get_history_by_round` 工具来获取指定轮数的历史信息。\",\n\"action\": \"get_history_by_round\",\n\"action_input\": {\n\"round\": 1\n},\n\"observation\": {}\n}\n```"
}
